# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.123122e+02     1.616637e+00
 * time: 0.0895850658416748
     1     1.048927e+01     9.493192e-01
 * time: 1.278975009918213
     2    -1.119111e+01     1.128007e+00
 * time: 1.408926010131836
     3    -3.403947e+01     9.001623e-01
 * time: 1.7196180820465088
     4    -4.736418e+01     6.827396e-01
 * time: 2.188441038131714
     5    -5.687638e+01     2.374132e-01
 * time: 2.3380930423736572
     6    -5.977187e+01     1.763838e-01
 * time: 2.4434170722961426
     7    -6.095461e+01     6.941153e-02
 * time: 2.558004140853882
     8    -6.143496e+01     5.603413e-02
 * time: 2.6736600399017334
     9    -6.168247e+01     5.296009e-02
 * time: 2.779266119003296
    10    -6.187262e+01     2.283531e-02
 * time: 2.8925909996032715
    11    -6.200089e+01     1.816388e-02
 * time: 3.0054991245269775
    12    -6.207527e+01     1.823870e-02
 * time: 3.113211154937744
    13    -6.211890e+01     1.579124e-02
 * time: 3.2263970375061035

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671056
    AtomicLocal         -18.8557589
    AtomicNonlocal      14.8522582
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485351 
    Xc                  -19.3336813

    total               -62.261666457073
